In [1]:
import os
import numpy as np # linear algebra.
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv).
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Loading metadata input.
metal_excel_dir = "../input/audio-based-violence-detection-dataset/VSD.xlsx"
metadata = pd.read_excel(metal_excel_dir,sheet_name='read_dataset')

In [3]:
# Cleaning metadata input.
metadata = metadata.drop(metadata[metadata['File_segment_name'] == "NaN"].index)
metadata.shape

(341, 5)

In [4]:
metadata.head(10)

,File_segment_name,Duration,Violence_start,Violence_end,Violence_duration
0,angry_011,117.0,21.0,38.000,17.00
1,angry_011,117.0,40.0,55.000,15.00
2,angry_011,117.0,60.0,79.000,19.00
3,angry_011,117.0,85.0,95.000,10.00
4,angry_011,117.0,101.0,110.000,9.00
5,angry_012,32.0,0.0,0.020,0.02
6,angry_013,72.0,0.0,0.010,0.00
7,angry_014,50.0,0.0,0.012,0.00
8,angry_015,162.0,64.0,71.000,7.00
9,angry_015,162.0,89.0,100.000,11.00


In [5]:
# Creating directory paths to store violence and non-violence data.
violence_dir = "../violence_data"
non_violence_dir = "../non_violence_data"

In [6]:
# Creating Directories.
os.mkdir(violence_dir)
os.mkdir(non_violence_dir)

In [7]:
# We have segments of audio in our data, Hence importing pydub to process segments.
from pydub import AudioSegment

# Looping over all the records in metadata input.
for record in metadata.values:
    t1 = 1000 * record[2]
    t2 = 1000 * record[3]
    audio  = AudioSegment.from_wav(os.path.join("../input/audio-based-violence-detection-dataset/audios_VSD/audios_VSD", str(record[0]) + ".wav"))
    violence_segment = audio[t1:t2]
    # Storing the violence audio segment in the corresponding directory.
    violence_segment.export(os.path.join(violence_dir,record[0]+".wav"))
    
print("===== Violent audio segments separated =====")

===== Violent audio segments separated =====


In [8]:
for record in metadata.values:
    t1 = 1000 * record[2]
    t2 = 1000 * record[3]
    audio  = AudioSegment.from_wav(os.path.join("../input/audio-based-violence-detection-dataset/audios_VSD/audios_VSD", str(record[0]) + ".wav"))
    non_violence_segment = audio[0:t1]
    non_violence_segment.export(os.path.join(non_violence_dir,record[0]+"_non_violence.wav"))

print("===== Non-violent audio segments separated =====")

===== Non-violent audio segments separated =====


In [9]:
# Importing Librosa to extract audio features.
import librosa

# Function to extract MFCC (Mel Frequency Cepstral Coefficients) from the audio sample.
def features_extractor(file):
    audio, sample_rate = librosa.load(file) 
    mfccs_features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
    # We do mean of transpose of value to find out scaled feature.
    mfccs_scaled_features = np.mean(mfccs_features.T,axis=0)
    return mfccs_scaled_features

In [10]:
# Creating empty list to store the extracted features.
extracted_features=[]

# Loop to process violent data extraction process.
for record in metadata.values:
    file_name = os.path.join(os.path.abspath(violence_dir), record[0] + ".wav")
    # Labelling violent audio as 1.
    final_class_labels = 1 
    try:
        data = features_extractor(file_name)
    except:
        continue
    extracted_features.append([data,final_class_labels])
print("===== Successfully extracted violent audio data =====")

# Loop to process non-violent data extraction process.
for record in metadata.values:
    file_name = os.path.join(os.path.abspath(non_violence_dir),record[0]+"_non_violence.wav")
    # Labelling non-violent audio as 0.
    final_class_labels = 0
    try:
        data = features_extractor(file_name)
    except:
        continue
    extracted_features.append([data,final_class_labels])
print("===== Successfully extracted non-violent audio data =====")

===== Successfully extracted violent audio data =====


[src/libmpg123/parse.c:do_readahead():1099] warning: Cannot read next header, a one-frame stream? Duh...
[src/libmpg123/parse.c:do_readahead():1099] warning: Cannot read next header, a one-frame stream? Duh...
[src/libmpg123/parse.c:do_readahead():1099] warning: Cannot read next header, a one-frame stream? Duh...
[src/libmpg123/parse.c:do_readahead():1099] warning: Cannot read next header, a one-frame stream? Duh...
[src/libmpg123/parse.c:do_readahead():1099] warning: Cannot read next header, a one-frame stream? Duh...
[src/libmpg123/parse.c:do_readahead():1099] warning: Cannot read next header, a one-frame stream? Duh...
[src/libmpg123/parse.c:do_readahead():1099] warning: Cannot read next header, a one-frame stream? Duh...
[src/libmpg123/parse.c:do_readahead():1099] warning: Cannot read next header, a one-frame stream? Duh...
[src/libmpg123/parse.c:do_readahead():1099] warning: Cannot read next header, a one-frame stream? Duh...
[src/libmpg123/parse.c:do_readahead():1099] warning: Ca

===== Successfully extracted non-violent audio data =====


[src/libmpg123/parse.c:do_readahead():1099] warning: Cannot read next header, a one-frame stream? Duh...


In [11]:
# Converting extracted_features to Pandas dataframe.
extracted_features_df = pd.DataFrame(extracted_features,columns=['feature','class'])
extracted_features_df.head()

,feature,class
0,"[-136.46625, 114.69909, -33.55017, 21.952213, ...",1
1,"[-136.46625, 114.69909, -33.55017, 21.952213, ...",1
2,"[-136.46625, 114.69909, -33.55017, 21.952213, ...",1
3,"[-136.46625, 114.69909, -33.55017, 21.952213, ...",1
4,"[-136.46625, 114.69909, -33.55017, 21.952213, ...",1


In [12]:
# Checking Value counts.
extracted_features_df["class"].value_counts()

class
1    341
0    315
Name: count, dtype: int64

In [13]:
# Split the dataset into independent and dependent dataset.
X = np.array(extracted_features_df["feature"].tolist())
y = np.array(extracted_features_df["class"].tolist())

# Train Test Split.
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.1,random_state=42)

In [14]:
print(X_train.shape, X_test.shape)

(590, 40) (66, 40)


In [15]:
# Reshaping to make the datasets compatible with LSTM.
X_train=X_train.reshape(590,1,40)
X_test=X_test.reshape(66,1, 40)
print(X_train.shape, X_test.shape)

(590, 1, 40) (66, 1, 40)


In [16]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Activation,Flatten,SimpleRNN, LSTM
from tensorflow.keras.optimizers import Adam
from sklearn import metrics

# No of classes.
num_labels=y.shape[0]
model=Sequential()
model.add(LSTM(64,input_shape=(1,40)))

### 1st layer.
model.add(Dense(32))
#model.add(Activation('relu'))
#model.add(Dropout(0.5))

### 2nd layer.
#model.add(Dense(128))
#model.add(Activation('relu'))
#model.add(Dropout(0.4))

### 3rd layer.
#model.add(Dense(64))
#model.add(Activation('relu'))
#model.add(Dropout(0.5))

### 4th layer.
#model.add(Dense(32))
#model.add(Activation('relu'))

### 5th layer.
#model.add(Dense(16))
#model.add(Activation('relu'))

### 6th layer.
#model.add(Dense(8))
#model.add(Activation('relu'))

### Final layer.
model.add(Dense(1))
model.add(Activation('sigmoid'))

In [17]:
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 64)                26880     
                                                                 
 dense (Dense)               (None, 32)                2080      
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
 activation (Activation)     (None, 1)                 0         
                                                                 
Total params: 28,993
Trainable params: 28,993
Non-trainable params: 0
_________________________________________________________________


In [18]:
# Model Compilation.
model.compile(loss='binary_crossentropy',metrics=['accuracy'],optimizer='adam')

In [19]:
# Trianing model.
from tensorflow.keras.callbacks import ModelCheckpoint
from datetime import datetime 
num_epochs = 10
num_batch_size = 32
checkpointer = ModelCheckpoint(filepath='./audio_classification.hdf5', verbose=1, save_best_only=True)
start = datetime.now()
model.fit(X_train, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(X_test, y_test), callbacks=[checkpointer], verbose=1)
duration = datetime.now() - start
print("Training completed in time:", duration)

Epoch 1/10
17/19 [=========================>....] - ETA: 0s - loss: 0.6896 - accuracy: 0.5129 
Epoch 1: val_loss improved from inf to 0.65128, saving model to ./audio_classification.hdf5
19/19 [==============================] - 4s 44ms/step - loss: 0.6868 - accuracy: 0.5271 - val_loss: 0.6513 - val_accuracy: 0.5909
Epoch 2/10
15/19 [======================>.......] - ETA: 0s - loss: 0.5926 - accuracy: 0.7250
Epoch 2: val_loss improved from 0.65128 to 0.57414, saving model to ./audio_classification.hdf5
19/19 [==============================] - 0s 7ms/step - loss: 0.5926 - accuracy: 0.7169 - val_loss: 0.5741 - val_accuracy: 0.6364
Epoch 3/10
16/19 [========================>.....] - ETA: 0s - loss: 0.5131 - accuracy: 0.7520
Epoch 3: val_loss improved from 0.57414 to 0.53754, saving model to ./audio_classification.hdf5
19/19 [==============================] - 0s 7ms/step - loss: 0.5190 - accuracy: 0.7441 - val_loss: 0.5375 - val_accuracy: 0.6212
Epoch 4/10
18/19 [===========================

In [20]:
# Evaluating the model on testing data.
test_accuracy = model.evaluate(X_test,y_test,verbose=0)
print("Test accuracy of the model:", test_accuracy[1])

Test accuracy of the model: 0.8636363744735718


In [21]:
# Evaluating the model on training data.
train_accuracy = model.evaluate(X_train,y_train,verbose=0)
print("Train accuracy of the model:", train_accuracy[1])

Train accuracy of the model: 0.9440677762031555


In [24]:
# Prediction on test data.
predict_x = model.predict(X_test)
print(predict_x)
#classes_x = np.argmax(predict_x,axis=1)
#print(classes_x)

3/3 [==============================] - 0s 3ms/step
[[0.9421848 ]
 [0.99350417]
 [0.5448032 ]
 [0.17477724]
 [0.10112327]
 [0.86577976]
 [0.46076092]
 [0.99350417]
 [0.6018717 ]
 [0.7599562 ]
 [0.95611954]
 [0.9891732 ]
 [0.20170519]
 [0.03494135]
 [0.45399684]
 [0.00870805]
 [0.17930932]
 [0.45399684]
 [0.12646396]
 [0.7599562 ]
 [0.4192578 ]
 [0.38130635]
 [0.29285124]
 [0.08705474]
 [0.06707651]
 [0.01746246]
 [0.8780639 ]
 [0.06707651]
 [0.15797989]
 [0.98899037]
 [0.99726665]
 [0.96849877]
 [0.95611954]
 [0.83749014]
 [0.99726665]
 [0.95086807]
 [0.08705474]
 [0.2523387 ]
 [0.8235343 ]
 [0.9891732 ]
 [0.17477724]
 [0.24072577]
 [0.78124034]
 [0.7599562 ]
 [0.1367127 ]
 [0.19020621]
 [0.23360994]
 [0.6549962 ]
 [0.981853  ]
 [0.913608  ]
 [0.19091396]
 [0.5341393 ]
 [0.00870805]
 [0.9753863 ]
 [0.9943231 ]
 [0.23018786]
 [0.8151447 ]
 [0.8452584 ]
 [0.38130635]
 [0.12646396]
 [0.99991333]
 [0.9977903 ]
 [0.99991333]
 [0.3721704 ]
 [0.9508681 ]
 [0.12646404]]


In [25]:
# Saving the model.
model.save("model_LSTM_v1.h5")